In [1]:
from frankwolfe_optimizer import (
    FrankWolfeOptimizer, ConvexProblem, print_fw_solution
)
import numpy as np

In [2]:
def example_1_quadratic_box():
    print("\n" + "*"*70)
    print("EXAMPLE 1: QUADRATIC OPTIMIZATION WITH BOX CONSTRAINTS")
    print("*"*70)
    
    n = 5
    
    # Create positive definite matrix A
    np.random.seed(42)
    M = np.random.randn(n, n)
    A = M.T @ M + np.eye(n)  # Ensure positive definite
    
    # Linear term
    b = np.random.randn(n)
    
    print(f"\nProblem dimension: {n}")
    print(f"Constraints: 0 ≤ x_i ≤ 5 for all i")
    
    # Setup problem
    problem = ConvexProblem()
    problem.set_quadratic_objective(A, b)
    problem.set_box_constraints(lower=np.zeros(n), upper=5*np.ones(n))
    problem.set_initial_point(2.5 * np.ones(n))  # Start at center
    
    # Solve
    result = problem.solve(max_iterations=100, verbose=True)
    print_fw_solution(result, "Quadratic with Box Constraints")
    
    return result


def example_2_portfolio_optimization():
    print("\n" + "*"*70)
    print("EXAMPLE 2: PORTFOLIO OPTIMIZATION (MARKOWITZ MODEL)")
    print("*"*70)
    
    n = 10  # Number of assets
    
    # Generate covariance matrix (risk)
    np.random.seed(123)
    M = np.random.randn(n, n)
    Sigma = M.T @ M / n  # Covariance matrix
    
    # Expected returns
    mu = np.random.uniform(0.05, 0.15, n)
    
    # Risk aversion parameter
    lambda_risk = 2.0
    
    # Objective: minimize lambda*risk - return
    A = lambda_risk * Sigma
    b = -mu
    
    print(f"\nNumber of assets: {n}")
    print(f"Risk aversion parameter: {lambda_risk}")
    print(f"Expected returns range: [{mu.min():.4f}, {mu.max():.4f}]")
    
    # Setup problem
    problem = ConvexProblem()
    problem.set_quadratic_objective(A, b)
    problem.set_simplex_constraint(n, radius=1.0)
    problem.set_initial_point(np.ones(n) / n)  # Equal weights
    
    # Solve
    result = problem.solve(max_iterations=200, tolerance=1e-8, verbose=True)
    print_fw_solution(result, "Portfolio Optimization")
    
    # Additional portfolio statistics
    x_opt = result['solution']
    portfolio_return = np.dot(mu, x_opt)
    portfolio_risk = np.sqrt(np.dot(x_opt, np.dot(Sigma, x_opt)))
    
    print("Portfolio Statistics:")
    print(f"  Expected Return: {portfolio_return:.4f}")
    print(f"  Risk (Std Dev): {portfolio_risk:.4f}")
    print(f"  Number of assets held: {np.sum(x_opt > 1e-6)}")
    print(f"  Largest position: {x_opt.max():.4f}")
    
    return result


def example_3_lasso_regression():
    print("\n" + "*"*70)
    print("EXAMPLE 3: LASSO REGRESSION (L1 REGULARIZATION)")
    print("*"*70)
    
    # Generate synthetic data
    np.random.seed(456)
    m, n = 50, 20  # m samples, n features
    A = np.random.randn(m, n)
    x_true = np.random.randn(n)
    x_true[n//2:] = 0  # Sparse ground truth
    b = A @ x_true + 0.1 * np.random.randn(m)
    
    # Objective: minimize ||Ax - b||²
    Q = A.T @ A
    c = -A.T @ b
    
    # L1 constraint radius
    R = 5.0
    
    print(f"\nData: {m} samples, {n} features")
    print(f"L1 ball radius: {R}")
    print(f"True solution sparsity: {np.sum(np.abs(x_true) > 1e-6)}/{n}")
    
    # Setup problem
    problem = ConvexProblem()
    problem.set_quadratic_objective(Q, c)
    problem.set_l1_ball_constraint(n, radius=R)
    problem.set_initial_point(np.zeros(n))
    
    # Solve
    result = problem.solve(max_iterations=300, verbose=True)
    print_fw_solution(result, "LASSO Regression")
    
    # Compare with true solution
    x_opt = result['solution']
    print("Solution Statistics:")
    print(f"  Solution sparsity: {np.sum(np.abs(x_opt) > 1e-3)}/{n}")
    print(f"  ||x_opt||₁: {np.sum(np.abs(x_opt)):.4f}")
    print(f"  Training error: {np.linalg.norm(A @ x_opt - b):.4f}")
    
    return result


def example_4_matrix_completion():
    print("\n" + "*"*70)
    print("EXAMPLE 4: VECTOR OPTIMIZATION WITH L2 BALL")
    print("*"*70)
    
    n = 30
    
    # Quadratic objective
    np.random.seed(789)
    M = np.random.randn(n, n)
    A = M.T @ M + 2*np.eye(n)
    b = np.random.randn(n)
    
    # L2 ball constraint
    radius = 3.0
    
    print(f"\nProblem dimension: {n}")
    print(f"Constraint: ||x||₂ ≤ {radius}")
    
    # Setup problem
    problem = ConvexProblem()
    problem.set_quadratic_objective(A, b)
    problem.set_l2_ball_constraint(n, radius=radius)
    problem.set_initial_point(np.zeros(n))
    
    # Solve
    result = problem.solve(max_iterations=150, verbose=True)
    print_fw_solution(result, "L2 Ball Constraint")
    
    x_opt = result['solution']
    print(f"Solution L2 norm: {np.linalg.norm(x_opt):.6f}")
    
    return result


def example_5_custom_objective():
    """
    Example 5: Custom Non-Quadratic Objective
    
    Minimize: sum(exp(x_i)) + ||x||²
    Subject to: 0 ≤ x ≤ 2
    """
    print("\n" + "*"*70)
    print("EXAMPLE 5: CUSTOM NON-QUADRATIC OBJECTIVE")
    print("*"*70)
    
    n = 8
    
    # Custom objective: sum(exp(x_i)) + ||x||²
    def objective(x):
        return np.sum(np.exp(x)) + np.dot(x, x)
    
    def gradient(x):
        return np.exp(x) + 2*x
    
    print(f"\nObjective: f(x) = Σ exp(x_i) + ||x||²")
    print(f"Dimension: {n}")
    print(f"Constraints: 0 ≤ x_i ≤ 2")
    
    # Setup problem
    problem = ConvexProblem()
    problem.set_custom_objective(objective, gradient)
    problem.set_box_constraints(lower=np.zeros(n), upper=2*np.ones(n))
    problem.set_initial_point(np.ones(n))
    
    # Solve
    result = problem.solve(max_iterations=200, line_search='backtracking', verbose=True)
    print_fw_solution(result, "Custom Objective")
    
    return result


def example_6_convergence_rates():
    print("\n" + "*"*70)
    print("EXAMPLE 6: COMPARING LINE SEARCH METHODS")
    print("*"*70)
    
    n = 10
    
    # Setup a standard problem
    np.random.seed(999)
    M = np.random.randn(n, n)
    A = M.T @ M + np.eye(n)
    b = np.random.randn(n)
    
    print(f"\nProblem: Quadratic with box constraints")
    print(f"Dimension: {n}")
    
    methods = ['exact', 'backtracking', 'fixed']
    results = {}
    
    for method in methods:
        print(f"\n--- Line Search: {method.upper()} ---")
        
        problem = ConvexProblem()
        problem.set_quadratic_objective(A, b)
        problem.set_box_constraints(lower=-2*np.ones(n), upper=2*np.ones(n))
        problem.set_initial_point(np.zeros(n))
        
        result = problem.solve(
            max_iterations=100,
            line_search=method,
            verbose=False
        )
        results[method] = result
        
        print(f"Converged: {result['converged']}")
        print(f"Iterations: {result['iterations']}")
        print(f"Final Gap: {result['final_gap']:.6e}")
        print(f"Optimal Value: {result['optimal_value']:.6f}")
    
    print("\n" + "*"*70)
    print("COMPARISON SUMMARY")
    print("*"*70)
    for method in methods:
        r = results[method]
        print(f"{method.upper():<15} Iters: {r['iterations']:<5} "
              f"Gap: {r['final_gap']:.2e}  Value: {r['optimal_value']:.6f}")
    
    return results


def example_7_high_dimensional():
    """
    Example 7: High-Dimensional Sparse Problem
    
    Demonstrates Frank-Wolfe on larger problems.
    """
    print("\n" + "*"*70)
    print("EXAMPLE 7: HIGH-DIMENSIONAL SPARSE OPTIMIZATION")
    print("*"*70)
    
    n = 100
    
    # Create sparse problem
    np.random.seed(555)
    A = np.diag(np.random.uniform(0.1, 2.0, n))  # Diagonal (sparse)
    b = np.random.randn(n)
    
    print(f"\nProblem dimension: {n}")
    print(f"Constraint: Probability simplex (sum = 1, x ≥ 0)")
    
    # Setup problem
    problem = ConvexProblem()
    problem.set_quadratic_objective(A, b)
    problem.set_simplex_constraint(n, radius=1.0)
    problem.set_initial_point(np.ones(n) / n)
    
    # Solve
    result = problem.solve(max_iterations=500, tolerance=1e-8, verbose=True)
    
    # Summary without printing full solution
    print("\n" + "*"*70)
    print(f"HIGH-DIMENSIONAL SOLUTION SUMMARY")
    print("*"*70)
    print(f"\nStatus: {'CONVERGED' if result['converged'] else 'MAX ITERATIONS'}")
    print(f"Iterations: {result['iterations']}")
    print(f"Final Gap: {result['final_gap']:.6e}")
    print(f"Optimal Value: {result['optimal_value']:.6f}")
    
    x_opt = result['solution']
    print(f"\nSolution sparsity: {np.sum(x_opt > 1e-6)}/{n} non-zeros")
    print(f"Largest components: {sorted(x_opt, reverse=True)[:5]}")
    print("*"*70)
    
    return result


def run_all_examples():
    examples = [
        ("Quadratic Box", example_1_quadratic_box),
        ("Portfolio Optimization", example_2_portfolio_optimization),
        ("LASSO Regression", example_3_lasso_regression),
        ("L2 Ball Constraint", example_4_matrix_completion),
        ("Custom Objective", example_5_custom_objective),
        ("Line Search Comparison", example_6_convergence_rates),
        ("High-Dimensional", example_7_high_dimensional),
    ]
    
    results = {}
    for name, example_func in examples:
        try:
            print(f"\n\n{'='*70}")
            print(f"Running: {name}")
            print(f"{'='*70}")
            result = example_func()
            results[name] = result
        except Exception as e:
            print(f"\nError in {name}: {str(e)}")
            results[name] = None
        
        input("\nPress Enter to continue to next example...")
    
    # Final summary
    print("\n" + "*"*70)
    print("TEST SUITE SUMMARY")
    print("*"*70)
    
    for name, result in results.items():
        if result is not None:
            if isinstance(result, dict) and 'converged' in result:
                status = "✓ CONVERGED" if result['converged'] else "○ MAX ITER"
                iters = result.get('iterations', 'N/A')
                print(f"{name:<30} {status:<15} Iterations: {iters}")
            else:
                print(f"{name:<30} ✓ COMPLETED")
        else:
            print(f"{name:<30} ✗ FAILED")
    
    print("*"*70)
    
    successful = sum(1 for r in results.values() 
                    if r is not None and (not isinstance(r, dict) or r.get('converged', False)))
    print(f"\nTotal Examples: {len(examples)}")
    print(f"Successful: {successful}")
    print("*"*70)


if __name__ == "__main__":
    run_all_examples()



Running: Quadratic Box

**********************************************************************
EXAMPLE 1: QUADRATIC OPTIMIZATION WITH BOX CONSTRAINTS
**********************************************************************

Problem dimension: 5
Constraints: 0 ≤ x_i ≤ 5 for all i
Frank-Wolfe Algorithm
Iter    f(x)              Gap               Step      
----------------------------------------------------------------------
0       1.430324e+02      --                --        
1       -2.577149e-02     2.899566e+02      0.9868    
11      -1.238553e-01     5.490188e-01      0.0050    
21      -1.297569e-01     1.587158e-02      0.0547    
31      -1.323747e-01     1.258001e-02      0.0435    
41      -1.341014e-01     1.043619e-02      0.0361    
51      -1.353306e-01     8.931309e-03      0.0310    
61      -1.362529e-01     7.815136e-03      0.0271    
71      -1.369720e-01     6.952820e-03      0.0242    
81      -1.375491e-01     6.265647e-03      0.0218    
91      -1.380231e-01 


Press Enter to continue to next example... 




Running: Portfolio Optimization

**********************************************************************
EXAMPLE 2: PORTFOLIO OPTIMIZATION (MARKOWITZ MODEL)
**********************************************************************

Number of assets: 10
Risk aversion parameter: 2.0
Expected returns range: [0.0503, 0.1488]
Frank-Wolfe Algorithm
Iter    f(x)              Gap               Step      
----------------------------------------------------------------------
0       1.899597e-02      --                --        
1       -2.236000e-02     4.279876e-01      0.1933    
11      -7.502550e-02     4.721529e-02      0.0224    
21      -7.809876e-02     3.082737e-02      0.0078    
31      -8.000488e-02     3.141719e-02      0.0080    
41      -8.092698e-02     1.709179e-02      0.0046    
51      -8.152514e-02     1.399441e-02      0.0068    
61      -8.196295e-02     1.115902e-02      0.0029    
71      -8.227125e-02     1.121042e-02      0.0055    
81      -8.252110e-02     8.514623e-


Press Enter to continue to next example... 




Running: LASSO Regression

**********************************************************************
EXAMPLE 3: LASSO REGRESSION (L1 REGULARIZATION)
**********************************************************************

Data: 50 samples, 20 features
L1 ball radius: 5.0
True solution sparsity: 10/20
Frank-Wolfe Algorithm
Iter    f(x)              Gap               Step      
----------------------------------------------------------------------
0       0.000000e+00      --                --        
1       -2.242859e+01     2.481108e+02      0.1808    
11      -8.498936e+01     5.363397e+01      0.0582    
21      -8.959411e+01     2.691927e+01      0.0197    
31      -9.191498e+01     1.784308e+01      0.0147    
41      -9.323081e+01     1.525966e+01      0.0154    
51      -9.408440e+01     1.112155e+01      0.0083    
61      -9.472417e+01     9.744017e+00      0.0121    
71      -9.519895e+01     8.866971e+00      0.0083    
81      -9.559202e+01     7.926479e+00      0.0093    
91

...



Running: Line Search Comparison

**********************************************************************
EXAMPLE 6: COMPARING LINE SEARCH METHODS
**********************************************************************

Problem: Quadratic with box constraints
Dimension: 10

--- Line Search: EXACT ---
Converged: False
Iterations: 100
Final Gap: 1.401617e+00
Optimal Value: -2.312628

--- Line Search: BACKTRACKING ---
Converged: False
Iterations: 100
Final Gap: 1.385222e+00
Optimal Value: -2.267257

--- Line Search: FIXED ---
Converged: False
Iterations: 100
Final Gap: 6.055513e+00
Optimal Value: -2.183790

**********************************************************************
COMPARISON SUMMARY
**********************************************************************
EXACT           Iters: 100   Gap: 1.40e+00  Value: -2.312628
BACKTRACKING    Iters: 100   Gap: 1.39e+00  Value: -2.267257
FIXED           Iters: 100   Gap: 6.06e+00  Value: -2.183790



Press Enter to continue to next example... 




Running: High-Dimensional

**********************************************************************
EXAMPLE 7: HIGH-DIMENSIONAL SPARSE OPTIMIZATION
**********************************************************************

Problem dimension: 100
Constraint: Probability simplex (sum = 1, x ≥ 0)
Frank-Wolfe Algorithm
Iter    f(x)              Gap               Step      
----------------------------------------------------------------------
0       -1.187547e-01     --                --        
1       -2.283489e+00     2.395232e+00      1.0000    
11      -2.419797e+00     1.166049e-06      0.0000    

Converged! Duality gap = 5.29e-09 < 1.00e-08

**********************************************************************
HIGH-DIMENSIONAL SOLUTION SUMMARY
**********************************************************************

Status: CONVERGED
Iterations: 16
Final Gap: 5.289774e-09
Optimal Value: -2.419797

Solution sparsity: 3/100 non-zeros
Largest components: [0.42968850827236177, 0.333618060


Press Enter to continue to next example... 



**********************************************************************
TEST SUITE SUMMARY
**********************************************************************
Quadratic Box                  ○ MAX ITER      Iterations: 100
Portfolio Optimization         ○ MAX ITER      Iterations: 200
LASSO Regression               ○ MAX ITER      Iterations: 300
L2 Ball Constraint             ✓ CONVERGED     Iterations: 141
Custom Objective               ✓ CONVERGED     Iterations: 2
Line Search Comparison         ✓ COMPLETED
High-Dimensional               ✓ CONVERGED     Iterations: 16
**********************************************************************

Total Examples: 7
Successful: 3
**********************************************************************
